# Lecture 9: Methods using KKT conditions

## Sequential Quadratic Programming (SQP)

**Idea is to generate a sequence of quadratic optimization problems whose solutions approach the solution of the original problem**

Let us consider problem
$$
\min f(x)\\
\text{s.t. }h_k(x) = 0\text{ for all }k=1,\ldots,K,
$$
where the the objective function and the equality constraints are twice differentiable.

Because we know that the optimal solution of this problem satisfies the KKT conditions, we know that
$$
\left\{\begin{array}{l}
\nabla_xL(x,\lambda,\mu)=\nabla f(x) + \mu\nabla h(x) = 0\\
h(x) = 0
\end{array}\right.
$$
Let us assume that we have a current esimation for the solution of the equality constraints $(x^k,\mu^k)$, then according to the Newton's method for root finding (see e.g., https://en.wikipedia.org/wiki/Newton's_method), we have another solution $(x^k,\mu^k)^T+(p,v)^T$ of the problem by solving system of equations
$$
HL(x,\lambda,\mu)\left[\begin{align}p^T\\v^T\end{align}\right] = -\nabla L(x,\lambda,\mu).
$$
This can be written as
$$
\left[
\begin{array}{cc}
H_xL(x^k,\lambda,\mu^k)&\nabla h(x^k)\\
\nabla h(x^k)^T & 0
\end{array}
\right]
\left[\begin{array}{c}p^T\\v^T\end{array}\right] =
\left[
\begin{array}{c}
-\nabla_x L(x^k,\lambda,\mu^k)\\
-h(x^k)^T
\end{array}
\right].
$$

However, the above is just the solution of the quadratic problem with equality constraints
$$
\min \frac12 p^TH_xL(x^k,\lambda,\mu^k)p+\nabla_xL(x^k,\lambda,\mu^k)^Tp\\
\text{s.t. }h_j(x^k) + \nabla h_j(x^k)^Tp = 0. 
$$

## Intuitive interpretation

We are approximating the function quadratically around the current solution and the constraints are approximated linearly.

## Implementation

Define an optimization problem, where
* $f(x) = \|x\|^2$
* $h(x) = \sum_{i=1}^nx_i-n$

In [1]:
def f_constrained(x):
    return sum([i**2 for i in x]),[],[sum(x)-len(x)]

In [3]:
print f_constrained([1,0,1])
print f_constrained([1,2,3,4])

(2, [], [-1])
(30, [], [6])


In [12]:
import numpy as np
import ad

def L(f,x,m):
    return f(x)[0]+np.matrix(m)*np.matrix(f(x)[2]).transpose()


def solve_QP(f,x,m):
    left_side_first_row = np.concatenate((\
    np.matrix(ad.gh(lambda y: L(f,y,m))[1](x)),\
    np.matrix(ad.gh(lambda y: f(y)[2])[0](x))).transpose(),axis=1)
    left_side_second_row = np.concatenate((\
    np.matrix(ad.gh(lambda y: f(y)[2])[0](x)),\
    np.matrix(0)),axis=1)
    right_hand_side = np.concatenate((\
    -1*np.matrix(ad.gh(lambda y: L(f,y,m))[0](x)).transpose(),
    -np.matrix(f(x)[2]).transpose()),axis = 0)
    left_hand_side = np.concatenate((\
                                    left_side_first_row,\
                                    left_side_second_row),axis = 0)
    temp = np.linalg.solve(left_hand_side,right_hand_side)
    return temp[:len(x)],temp[len(x):]
    
    

def SQP(f,start,precision):
    x = start
    m = np.ones(len(start))
    f_old = float('inf')
    f_new = f(x)[0]
    while abs(f_old-f_new)>precision:
        f_old = f_new
        (p,v) = solve_QP(f,x,m)
        x = x+p
        m = m+v
        f_new = f(x)[0]
    return x

In [13]:
SQP(f_constrained,[0,0],0.001)

ValueError: shapes (1,2) and (1,1) not aligned: 2 (dim 1) != 1 (dim 0)

## Lagrangian methods -- "The original method of multipliers"

Let us again consider problem
$$
\min f(x)\\
\text{s.t. }h_k(x) = 0\text{ for all }k=1,\ldots,K,
$$
where the the objective function and the equality constraints are twice differentiable.

Define augmented Lagrangian function
$$
L_c(x,\mu) = f(x)+\mu h(x)+\frac12c\|h(x)\|^2.
$$
Above $c\in \mathbb R$ is a penalty parameter and $\mu \in \mathbb R^n$ is a multiplier.

Let us consider sequence of optimization problems
$$
\min_{x\in\mathbb R^n} f(x)+\mu_k h(x)+\frac{1}{2}c_k\|h(x)\|^2,
$$
where $c_{k+1}>c_k$ for $k=1,2,\ldots$.

Now, if $\mu_k=0$ for all $k=1,2,\ldots$, then we have a penalty function method, which solves the problem when $c_k\to \infty$.

However, it can be shown, that if we set $\mu_0$ randomly and keep on updating
$\mu_{k+1} = \mu_k-c_kh(x)$, then we can show that there exists $C>0$ such that of $c_k>C$, then the optimal solution of the augmented Langrangian solves the original problem!

### Example

Let us have optimization problem
$$
\min x_1^2+x_2^2\\
\text{s.t. }x_1+x_2-1=0.
$$

Now, the minimizatio of the augmented Lagrangian becomes
$$
\min_{x\in\mathbb R^n} x_1^2+x_2^2+\mu_k(x_1+x_2-1)+\frac12c_k(x_1+x_2-1)^2.\\
$$

In [14]:
def f_constrained2(x):
    return sum([i**2 for i in x]),[],[sum(x)-1]

In [15]:
def augmented_langrangian(f,x,mu,c):
    second_term = float(numpy.matrix(mu)*numpy.matrix(f(x)[2]).transpose())
    third_term = 0.5*c*numpy.linalg.norm(f(x)[2])**2
    return f(x)[0]-second_term+third_term

In [16]:
from scipy.optimize import minimize
import numpy
def augmented_langrangian_method(f,start,mu0,c0):
    x_old = [float('inf')]*2
    x_new = start
    mu = mu0
    c = c0
    while numpy.linalg.norm(f(x_new)[2])>0.00001:
        res = minimize(lambda x:augmented_langrangian(f,x,mu,c),x_new)
        x_old = x_new
        mu = float(mu-numpy.matrix(c)*numpy.matrix(f(x_old)[2]).transpose())
        x_new = res.x
        c = 2*c
    return x_new,c

In [17]:
from scipy.optimize import minimize
import numpy
def penalty_function_method(f,start,c0):
    x_old = [float('inf')]*2
    x_new = start
    c = c0
    while numpy.linalg.norm(f(x_new)[2])>0.00001:
        res = minimize(lambda x:augmented_langrangian(f,x,0,c),x_new)
        x_old = x_new
        x_new = res.x
        c = 2*c
    return x_new,c

In [18]:
augmented_langrangian_method(f_constrained2,[0,0],1,1)

(array([ 0.5,  0.5]), 2)

In [19]:
penalty_function_method(f_constrained2,[0,0],1)

(array([ 0.49999618,  0.49999618]), 262144)

## What is going on in here?

This is not completely trivial, unfortunately. If you want to read details, please see e.g., http://www.mit.edu/~dimitrib/Constrained-Opt.pdf.